In [1]:
from eval.stt import evaluate_asr_model

references = ["hello world", "test sentence"]
hypotheses = ["hello word", "test sentance"]

result = evaluate_asr_model(references, hypotheses, language="en")
print(f"WER: {result.wer:.4f}")


WER: 0.5000


In [4]:
result.wer_rubric.rubric_score


1

In [3]:
from eval.stt import evaluate_asr_model

references = ["hello world test sentence"]
hypotheses = ["hello world test sentance"]

result = evaluate_asr_model(references, hypotheses, language="en")
print(f"WER: {result.wer:.4f}")


WER: 0.2500


In [1]:
# Azure AI Foundry Integration
import json
import os
from pathlib import Path
from datetime import datetime
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Configuration for Azure AI Foundry
AZURE_AI_PROJECT_NAME = os.getenv("AZURE_AI_PROJECT_NAME", "ai-cap-upskilling-dev-sdc-482x")
AZURE_RESOURCE_GROUP = os.getenv("AZURE_RESOURCE_GROUP", "rg-cap-upskilling-dev-sdc")
AZURE_SUBSCRIPTION_ID = os.getenv("AZURE_SUBSCRIPTION_ID", "e1b63a44-9cef-40fa-8616-fdeea5b391cc")

In [14]:
def load_evaluation_results(results_dir="/workspaces/oryx-cap-upskilling/eval/results"):
    """Load all evaluation results from the results directory."""
    results_dir = Path(results_dir)
    evaluation_results = []
    
    for result_file in results_dir.glob("*/*.json"):
        try:
            with open(result_file, 'r') as f:
                result_data = json.load(f)
                result_data['result_file_path'] = str(result_file)
                evaluation_results.append(result_data)
                print(f"✓ Loaded: {result_file.name}")
        except Exception as e:
            print(f"✗ Error loading {result_file}: {e}")
    
    return evaluation_results

# Load evaluation results
eval_results = load_evaluation_results()
print(f"\nFound {len(eval_results)} evaluation result(s)")

# Display summary of results
for i, result in enumerate(eval_results, 1):
    print(f"\nResult {i}:")
    print(f"  Task: {result.get('task_type', 'unknown')}")
    print(f"  Timestamp: {result.get('timestamp', 'unknown')}")
    print(f"  Session ID: {result.get('session_id', 'unknown')}")
    if 'results' in result and 'wer' in result['results']:
        print(f"  WER: {result['results']['wer']:.4f}")
        print(f"  CER: {result['results']['cer']:.4f}")

In [15]:
def format_metrics_for_ai_foundry(eval_result):
    """Format evaluation results for Azure AI Foundry submission."""
    if 'results' not in eval_result:
        return {}
    
    results = eval_result['results']
    task_type = eval_result.get('task_type', 'unknown')
    
    # Format metrics based on task type
    formatted_metrics = {
        'task_type': task_type,
        'evaluation_timestamp': eval_result.get('timestamp'),
        'session_id': eval_result.get('session_id'),
    }
    
    if task_type == 'stt':
        # Speech-to-Text specific metrics
        formatted_metrics.update({
            'word_error_rate': results.get('wer', 0),
            'character_error_rate': results.get('cer', 0),
            'match_error_rate': results.get('mer', 0), 
            'word_information_preserved': results.get('wip', 0),
            'word_information_lost': results.get('wil', 0),
            'word_levenshtein_distance': results.get('word_levenshtein_distance', 0),
            'char_levenshtein_distance': results.get('char_levenshtein_distance', 0),
            'normalized_word_ld': results.get('normalized_word_ld', 0),
            'normalized_char_ld': results.get('normalized_char_ld', 0),
        })
        
        # Add rubric scores if available
        if 'wer_rubric' in results:
            formatted_metrics['wer_rubric_score'] = results['wer_rubric'].get('rubric_score', 0)
            formatted_metrics['wer_rubric_label'] = results['wer_rubric'].get('rubric_label', 'Unknown')
    
    elif task_type == 'diarization':
        # Diarization specific metrics  
        formatted_metrics.update({
            'diarization_error_rate': results.get('der', 0),
            'jaccard_error_rate': results.get('jer', 0),
            'missed_speech_ratio': results.get('missed_speech_ratio', 0),
            'false_alarm_ratio': results.get('false_alarm_ratio', 0),
            'speaker_confusion_ratio': results.get('speaker_confusion_ratio', 0),
        })
    
    return formatted_metrics

# Format results for Azure AI Foundry
formatted_results = []
for result in eval_results:
    formatted = format_metrics_for_ai_foundry(result)
    formatted_results.append(formatted)
    
print("Formatted results for Azure AI Foundry:")
for i, formatted in enumerate(formatted_results, 1):
    print(f"\nFormatted Result {i}:")
    for key, value in formatted.items():
        print(f"  {key}: {value}")

In [ ]:
class BlocklistEvaluator:
    def __init__(self, blocklist) -> None:  # noqa: ANN001
        self._blocklist = blocklist

    def __call__(self: "BlocklistEvaluator", *, response: str):  # noqa: ANN204
        score = any(word in self._blocklist for word in response.split())
        return {"score": score}

In [2]:
def test_custom_eval():
    return {"score": 10}

In [17]:
formatted_results[0]

{'task_type': 'stt',
 'evaluation_timestamp': '2025-11-10T11:03:25.177445',
 'session_id': '66e5e0e8-dc4a-403c-8bac-b85538b1cd19',
 'word_error_rate': 0.5,
 'character_error_rate': 0.13043478260869565,
 'match_error_rate': 0.5,
 'word_information_preserved': 0.25,
 'word_information_lost': 0.75,
 'word_levenshtein_distance': 1.0,
 'char_levenshtein_distance': 1.5,
 'normalized_word_ld': 0.5,
 'normalized_char_ld': 0.12878787878787878,
 'wer_rubric_score': 1,
 'wer_rubric_label': 'Poor'}

In [7]:
# Azure AI Foundry Integration
import json
import os
from pathlib import Path
from datetime import datetime
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
azure_ai_project = {
    "subscription_id": os.getenv("AZURE_SUBSCRIPTION_ID", "e1b63a44-9cef-40fa-8616-fdeea5b391cc"),
    "resource_group_name": os.getenv("AZURE_RESOURCE_GROUP", "rg-cap-upskilling-dev-sdc"),
    "project_name":  os.getenv("AZURE_AI_PROJECT_NAME", "aip-cap-upskilling-dev-sdc")
}

credential = DefaultAzureCredential()

In [18]:
class AnswerLengthEvaluator:
    def __init__(self):
        pass
    # A class is made callable by implementing the special method __call__
    def __call__(self, *, query: str, **kwargs):
        return {'task_type': 'stt',
                'evaluation_timestamp': '2025-11-10T11:03:25.177445',
                'session_id': '66e5e0e8-dc4a-403c-8bac-b85538b1cd19',
                'word_error_rate': 0.5,
                'character_error_rate': 0.13043478260869565,
                'match_error_rate': 0.5,
                'word_information_preserved': 0.25,
                'word_information_lost': 0.75,
                'word_levenshtein_distance': 1.0,
                'char_levenshtein_distance': 1.5,
                'normalized_word_ld': 0.5,
                'normalized_char_ld': 0.12878787878787878,
                'wer_rubric_score': 1,
                'wer_rubric_label': 'Poor'}
    
answer_length_evaluator = AnswerLengthEvaluator()
answer_length = answer_length_evaluator(query="What is the speed of light?")

In [ ]:
from azure.ai.evaluation import evaluate
azure_ai_project = "https://ai-cap-upskilling-dev-sdc-482x.services.ai.azure.com/api/projects/aip-cap-upskilling-dev-sdc"

results = evaluate(
    evaluation_name="stt_evaluation",
    data="data.jsonl",
    evaluators={
        "eval": answer_length_evaluator,
    },
    credential=credential,
    azure_ai_project = azure_ai_project,
)

In [12]:
credential = DefaultAzureCredential()
        
project_client = AIProjectClient(
            credential=credential,
            endpoint="https://ai-cap-upskilling-dev-sdc-482x.services.ai.azure.com/api/projects/aip-cap-upskilling-dev-sdc",
            subscription_id=AZURE_SUBSCRIPTION_ID,
            resource_group_name=AZURE_RESOURCE_GROUP,
            project_name= AZURE_AI_PROJECT_NAME
        )

In [13]:
project_client.evaluations

AttributeError: 'AIProjectClient' object has no attribute 'evaluations'

In [10]:
async def submit_to_azure_ai_foundry(formatted_results, project_name=None):
    """Submit evaluation results to Azure AI Foundry project.
    
    Args:
        formatted_results: List of formatted evaluation results
        project_name: Azure AI Foundry project name
    """
    
    try:
        print("🚀 Submitting to Azure AI Foundry...")
        print("="*50)
        
        # Initialize Azure AI Project Client
        credential = DefaultAzureCredential()
        
        project_client = AIProjectClient(
            credential=credential,
            subscription_id=AZURE_SUBSCRIPTION_ID,
            resource_group_name=AZURE_RESOURCE_GROUP,
            project_name=project_name or AZURE_AI_PROJECT_NAME
        )
        
        print(f"✅ Connected to Azure AI Project: {project_name or AZURE_AI_PROJECT_NAME}")
        
        # Submit each evaluation result
        submitted_evaluations = []
        
        for i, result in enumerate(formatted_results, 1):
            try:
                task_type = result.get('task_type', 'unknown')
                session_id = result.get('session_id', 'unknown')
                eval_name = f"{task_type}_evaluation_{session_id[:8]}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
                
                print(f"\n📊 Submitting Evaluation {i}/{len(formatted_results)}")
                print(f"   Name: {eval_name}")
                print(f"   Task: {task_type.upper()}")
                
                # Prepare evaluation data for Azure AI Foundry
                evaluation_data = {
                    "name": eval_name,
                    "description": f"Automated {task_type} evaluation submission",
                    "tags": {
                        "task_type": task_type,
                        "session_id": session_id,
                        "submission_timestamp": datetime.now().isoformat(),
                        "source": "oryx-evaluation-framework"
                    },
                    "properties": {
                        "evaluation_framework": "oryx-cap-upskilling",
                        "task_type": task_type,
                        "timestamp": result.get('evaluation_timestamp'),
                        "session_id": session_id
                    }
                }
                
                # Add metrics based on task type
                if task_type == 'stt':
                    evaluation_data["metrics"] = {
                        "word_error_rate": result.get('word_error_rate', 0),
                        "character_error_rate": result.get('character_error_rate', 0),
                        "match_error_rate": result.get('match_error_rate', 0),
                        "word_information_preserved": result.get('word_information_preserved', 0),
                        "word_information_lost": result.get('word_information_lost', 0),
                        "word_levenshtein_distance": result.get('word_levenshtein_distance', 0),
                        "char_levenshtein_distance": result.get('char_levenshtein_distance', 0),
                        "normalized_word_ld": result.get('normalized_word_ld', 0),
                        "normalized_char_ld": result.get('normalized_char_ld', 0),
                        "wer_rubric_score": result.get('wer_rubric_score', 0),
                        "wer_rubric_label": result.get('wer_rubric_label', 'Unknown')
                    }
                    print(f"   📈 WER: {result.get('word_error_rate', 0):.4f}")
                    print(f"   📈 CER: {result.get('character_error_rate', 0):.4f}")
                    print(f"   🎯 Rubric: {result.get('wer_rubric_label', 'Unknown')} ({result.get('wer_rubric_score', 0)})")
                
                elif task_type == 'diarization':
                    evaluation_data["metrics"] = {
                        "diarization_error_rate": result.get('diarization_error_rate', 0),
                        "jaccard_error_rate": result.get('jaccard_error_rate', 0),
                        "missed_speech_ratio": result.get('missed_speech_ratio', 0),
                        "false_alarm_ratio": result.get('false_alarm_ratio', 0),
                        "speaker_confusion_ratio": result.get('speaker_confusion_ratio', 0)
                    }
                    print(f"   ? DER: {result.get('diarization_error_rate', 0):.4f}")
                    print(f"   📈 JER: {result.get('jaccard_error_rate', 0):.4f}")
                
                # Submit to Azure AI Foundry using the Evaluations API
                try:
                    # Create evaluation run
                    evaluation_run = await project_client.evaluations.create_evaluation_run(
                        evaluation_name=eval_name,
                        description=evaluation_data["description"],
                        tags=evaluation_data["tags"],
                        properties=evaluation_data["properties"],
                        metrics=evaluation_data["metrics"]
                    )
                    
                    print(f"   ✅ Submitted successfully!")
                    print(f"   🆔 Evaluation ID: {evaluation_run.id}")
                    
                    submitted_evaluations.append({
                        "name": eval_name,
                        "id": evaluation_run.id,
                        "task_type": task_type,
                        "metrics": evaluation_data["metrics"]
                    })
                    
                except Exception as api_error:
                    # Fallback: Submit as custom metrics if direct evaluation API doesn't work
                    print(f"   ⚠️  Direct evaluation API failed, trying metrics logging...")
                    
                    # Log metrics using the project client's metrics logging
                    await project_client.telemetry.log_metrics(
                        run_name=eval_name,
                        metrics=evaluation_data["metrics"],
                        tags=evaluation_data["tags"],
                        properties=evaluation_data["properties"]
                    )
                    
                    print(f"   ✅ Metrics logged successfully!")
                    
                    submitted_evaluations.append({
                        "name": eval_name,
                        "task_type": task_type,
                        "metrics": evaluation_data["metrics"],
                        "method": "metrics_logging"
                    })
                
            except Exception as eval_error:
                print(f"   ❌ Failed to submit evaluation {i}: {eval_error}")
                continue
        
        # Summary
        print(f"\n{'='*50}")
        print(f"📋 Submission Summary")
        print(f"✅ Successfully submitted: {len(submitted_evaluations)}/{len(formatted_results)} evaluations")
        
        if submitted_evaluations:
            print(f"\n📊 Submitted Evaluations:")
            for eval_info in submitted_evaluations:
                print(f"   • {eval_info['name']} ({eval_info['task_type'].upper()})")
                if 'id' in eval_info:
                    print(f"     ID: {eval_info['id']}")
                if eval_info.get('method') == 'metrics_logging':
                    print(f"     Method: Metrics Logging")
        
        print(f"\n🌐 View your evaluations in Azure AI Foundry:")
        print(f"   https://ai.azure.com/projectOverview/subscriptions/{AZURE_SUBSCRIPTION_ID}/resourceGroups/{AZURE_RESOURCE_GROUP}/providers/Microsoft.MachineLearningServices/workspaces/{project_name or AZURE_AI_PROJECT_NAME}")
        
        return submitted_evaluations
        
    except Exception as e:
        print(f"❌ Error connecting to Azure AI Foundry: {e}")
        print(f"💡 Make sure you have:")
        print(f"   • Correct Azure credentials (run 'az login')")
        print(f"   • Valid subscription ID: {AZURE_SUBSCRIPTION_ID}")
        print(f"   • Valid resource group: {AZURE_RESOURCE_GROUP}")
        print(f"   • Valid project name: {project_name or AZURE_AI_PROJECT_NAME}")
        print(f"   • Required permissions on the Azure AI project")
        return []

# Submit the evaluation results
submission_results = await submit_to_azure_ai_foundry(formatted_results)

The history saving thread hit an unexpected error (UnicodeEncodeError('utf-8', 'async def submit_to_azure_ai_foundry(formatted_results, project_name=None):\n    """Submit evaluation results to Azure AI Foundry project.\n    \n    Args:\n        formatted_results: List of formatted evaluation results\n        project_name: Azure AI Foundry project name\n    """\n    \n    try:\n        print("🚀 Submitting to Azure AI Foundry...")\n        print("="*50)\n        \n        # Initialize Azure AI Project Client\n        credential = DefaultAzureCredential()\n        \n        project_client = AIProjectClient(\n            credential=credential,\n            subscription_id=AZURE_SUBSCRIPTION_ID,\n            resource_group_name=AZURE_RESOURCE_GROUP,\n            project_name=project_name or AZURE_AI_PROJECT_NAME\n        )\n        \n        print(f"✅ Connected to Azure AI Project: {project_name or AZURE_AI_PROJECT_NAME}")\n        \n        # Submit each evaluation result\n        submitte

UnicodeEncodeError: 'utf-8' codec can't encode character '\udcc8' in position 31: surrogates not allowed

## Azure AI Foundry Submission Configuration

### Prerequisites

Before running the submission, ensure you have:

1. **Azure CLI installed and authenticated:**
   ```bash
   az login
   ```

2. **Required Python packages:**
   ```bash
   pip install azure-ai-projects azure-identity
   ```

3. **Correct Azure configuration** (already set in cell 4):
   - `AZURE_SUBSCRIPTION_ID`: Your Azure subscription ID
   - `AZURE_RESOURCE_GROUP`: Your resource group name
   - `AZURE_AI_PROJECT_NAME`: Your Azure AI Foundry project name

4. **Azure permissions:**
   - Contributor or AI Developer role on the Azure AI project
   - Permission to create and manage evaluations

### What the submission does:

1. **Connects** to your Azure AI Foundry project using DefaultAzureCredential
2. **Creates evaluation runs** for each result with proper naming and metadata
3. **Submits metrics** including WER, CER, rubric scores, and other evaluation data
4. **Provides fallback** to metrics logging if direct evaluation API is unavailable
5. **Returns URLs** to view your evaluations in the Azure AI Foundry portal

### Troubleshooting:

- **Authentication errors**: Run `az login` and ensure you have access to the project
- **Permission errors**: Verify you have Contributor role on the Azure AI resource
- **Project not found**: Double-check the subscription ID, resource group, and project name
- **Network errors**: Ensure you have internet connectivity and Azure services are accessible

In [ ]:
# Prepare for Azure AI Foundry submission
def prepare_submission():
    """Final preparation and validation before Azure AI Foundry submission."""
    
    print("🔧 Pre-submission Checklist")
    print("="*40)
    
    # Check Azure configuration
    print("? Checking Azure Configuration:")
    print(f"   • Subscription ID: {AZURE_SUBSCRIPTION_ID}")
    print(f"   • Resource Group: {AZURE_RESOURCE_GROUP}")
    print(f"   • Project Name: {AZURE_AI_PROJECT_NAME}")
    
    # Check authentication
    try:
        from azure.identity import DefaultAzureCredential
        credential = DefaultAzureCredential()
        print("   ✅ Azure credentials available")
    except Exception as e:
        print(f"   ❌ Azure credential error: {e}")
        return False
    
    # Validate evaluation results
    print(f"\n📊 Evaluation Data:")
    print(f"   • Found {len(eval_results)} evaluation result(s)")
    print(f"   • Formatted {len(formatted_results)} result(s) for submission")
    
    for i, result in enumerate(formatted_results, 1):
        task_type = result.get('task_type', 'unknown')
        print(f"   • Result {i}: {task_type.upper()} evaluation")
        
        if task_type == 'stt':
            wer = result.get('word_error_rate', 0)
            print(f"     - WER: {wer:.4f}")
    
    print("\n✅ Ready for submission to Azure AI Foundry!")
    print("💡 Run the next cell to submit your evaluations")
    
    return True

# Run preparation check
is_ready = prepare_submission()

In [ ]:
# Verify the submission results
print("🔍 Submission Verification")
print("="*40)

if submission_results:
    print(f"✅ {len(submission_results)} evaluation(s) submitted successfully!")
    
    print("\n? Submitted Evaluations Detail:")
    for i, eval_info in enumerate(submission_results, 1):
        print(f"\n{i}. {eval_info['name']}")
        print(f"   Task Type: {eval_info['task_type'].upper()}")
        
        if 'id' in eval_info:
            print(f"   Azure ID: {eval_info['id']}")
        
        # Show key metrics
        metrics = eval_info.get('metrics', {})
        if eval_info['task_type'] == 'stt':
            wer = metrics.get('word_error_rate', 0)
            cer = metrics.get('character_error_rate', 0)
            rubric = metrics.get('wer_rubric_label', 'Unknown')
            print(f"   Key Metrics: WER={wer:.4f}, CER={cer:.4f}, Rubric={rubric}")
        
        elif eval_info['task_type'] == 'diarization':
            der = metrics.get('diarization_error_rate', 0)
            jer = metrics.get('jaccard_error_rate', 0)
            print(f"   Key Metrics: DER={der:.4f}, JER={jer:.4f}")
    
    print(f"\n🌐 Access your evaluations at:")
    print(f"https://ai.azure.com/projectOverview/subscriptions/{AZURE_SUBSCRIPTION_ID}/resourceGroups/{AZURE_RESOURCE_GROUP}/providers/Microsoft.MachineLearningServices/workspaces/{AZURE_AI_PROJECT_NAME}")

else:
    print("❌ No evaluations were submitted successfully.")
    print("💡 Check the error messages above and verify your Azure configuration.")

print(f"\n✨ Azure AI Foundry submission complete!")

In [1]:
from eval.main import EvaluationOrchestrator
from pathlib import Path
orchestrator = EvaluationOrchestrator()
references_file = "/workspaces/oryx-cap-upskilling/eval/datasets/stt/references.txt"
hypotheses_file = "/workspaces/oryx-cap-upskilling/eval/datasets/stt/hypotheses.txt"

references = orchestrator._load_transcriptions_file(references_file)
hypotheses = orchestrator._load_transcriptions_file(hypotheses_file)
        
        # Step 4: Run STT evaluation
        
results = orchestrator.run_stt_evaluation(
            references=references,
            hypotheses=hypotheses,
            language="en",  # or "ar" for Arabic
            references_file=Path(references_file),  # For Azure ML logging
            hypotheses_file=Path(hypotheses_file)   # For Azure ML logging
        )

2025-11-13 07:29:45,132 - eval.config - INFO - Configuration loaded with log level: INFO
2025-11-13 07:29:45,133 - eval.main - INFO - workspace_name='R-ML-Allam-Test' resource_group='RG-Allam-test' subscription_id='e1b63a44-9cef-40fa-8616-fdeea5b391cc' experiment_name='cap-evaluation-experiments' enable_logging=True
2025-11-13 07:31:46,137 - azure.identity._credentials.environment - INFO - No environment configuration found.
2025-11-13 07:31:46,242 - azure.identity._credentials.managed_identity - INFO - ManagedIdentityCredential will use IMDS
2025-11-13 07:31:53,604 - azure.core.pipeline.policies.http_logging_policy - INFO - Request URL: 'https://westus-0.in.applicationinsights.azure.com//v2.1/track'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '1321'
    'Accept': 'application/json'
    'x-ms-client-request-id': 'd37dc1e6-c062-11f0-8b2c-0a8099d40b7b'
    'User-Agent': 'azsdk-python-azuremonitorclient/unknown Python/3.13.9 (Linux-

🏃 View run stt_20251113_073259_7d0d4094 at: https://eastus2.api.azureml.ms/mlflow/v2.0/subscriptions/e1b63a44-9cef-40fa-8616-fdeea5b391cc/resourceGroups/RG-Allam-test/providers/Microsoft.MachineLearningServices/workspaces/R-ML-Allam-Test/#/experiments/e6858ff2-a178-4d17-b580-8d62af350fa8/runs/14a9ad01-62ba-4a89-94e2-f5c90ce785e5
🧪 View experiment at: https://eastus2.api.azureml.ms/mlflow/v2.0/subscriptions/e1b63a44-9cef-40fa-8616-fdeea5b391cc/resourceGroups/RG-Allam-test/providers/Microsoft.MachineLearningServices/workspaces/R-ML-Allam-Test/#/experiments/e6858ff2-a178-4d17-b580-8d62af350fa8


2025-11-13 07:33:59,094 - azure.identity._internal.decorators - INFO - AzureCliCredential.get_token succeeded
2025-11-13 07:33:59,095 - azure.identity._credentials.default - INFO - DefaultAzureCredential acquired a token from AzureCliCredential
2025-11-13 07:33:59,721 - eval.main - INFO - ✓ Logged to cloud - Run ID: 14a9ad01-62ba-4a89-94e2-f5c90ce785e5


2025-11-13 07:34:00,739 - azure.core.pipeline.policies.http_logging_policy - INFO - Request URL: 'https://westus2-0.in.applicationinsights.azure.com//v2.1/track'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '14068'
    'Accept': 'application/json'
    'x-ms-client-request-id': '1f44e4ba-c063-11f0-9861-0a8099d40b7b'
    'User-Agent': 'azsdk-python-azuremonitorclient/unknown Python/3.13.9 (Linux-5.15.167.4-microsoft-standard-WSL2-aarch64-with-glibc2.39)'
A body is sent with the request
2025-11-13 07:34:01,133 - azure.core.pipeline.policies.http_logging_policy - INFO - Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; charset=utf-8'
    'Server': 'Microsoft-HTTPAPI/2.0'
    'Strict-Transport-Security': 'REDACTED'
    'X-Content-Type-Options': 'REDACTED'
    'Date': 'Thu, 13 Nov 2025 07:33:57 GMT'
2025-11-13 07:34:01,136 - azure.monitor.opentelemetry.exporter.export._base 